# Setup

## Importing libraries

In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4


cuda


## Dataset

In [2]:
with open('WOOz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


## Encoder Decoder Function

In [3]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)

print(decoded_hello)

hello


# Encoding Dataset

In [4]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,  0,
         0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0,
         1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33,
        50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25,
        38, 28,  1, 39, 30,  1, 39, 50,  9,  1])


In [6]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randInt(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    x, y = x.to(device), y.to(device)



In [7]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("When input is",context,"target is",target)

When input is tensor([80]) target is tensor(28)
When input is tensor([80, 28]) target is tensor(39)
When input is tensor([80, 28, 39]) target is tensor(42)
When input is tensor([80, 28, 39, 42]) target is tensor(39)
When input is tensor([80, 28, 39, 42, 39]) target is tensor(44)
When input is tensor([80, 28, 39, 42, 39, 44]) target is tensor(32)
When input is tensor([80, 28, 39, 42, 39, 44, 32]) target is tensor(49)
When input is tensor([80, 28, 39, 42, 39, 44, 32, 49]) target is tensor(1)


This is sequential however, as what a CPU does

CPU can perform a lot of complex operations really quickly but only sequentially.

A GPU can perform simple, not computationally complex tasks, but running parallely

We can hence take each of these blocks, stack them together and push to the GPU to scale training a lot

Let's say we have 4 blocks stacked. We represent this by a new hyperparameter, **Batch Size**

# Bigram Language Model

In [ ]:
class BigramLanguageModel(nn.Module):
    